# 导入包和函数

In [ ]:
import os
import sys
import time
from multiprocessing import Pool, cpu_count
import numpy as np

np.random.seed(756)

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

from mfdnn import *
from utils import *

# 基础设置

In [ ]:

data_path = "RunTimeAnalysis"

Ts = [32, 64, 128]
Ps = [10, 50, 100]
n = 200

configurations = [
    {'T': T, 'p': p}
    for T in Ts
    for p in Ps
]

lam1_values = [0.5, 1, 1.5, 2, 2.5, 3]
lam2_values = [0, 0.001, 0.01, 0.1, 0.5, 1]

model_params = {
    'num_basis': (5, 5),
    'layer_sizes': [64, 64],
    'epochs': 100,
    'val_ratio': 0.2,
    'patience': 10
}

true_vars = {0, 1}
epsilon = 0.01

# 辅助函数

In [ ]:
def calculate_selection_metrics(l21_norms, true_vars, epsilon=0.01, p=6):
    selected_vars = set(i for i, norm in enumerate(l21_norms) if norm > epsilon)

    true_positive = len(selected_vars & true_vars)
    false_positive = len(selected_vars - true_vars)
    false_negative = len(true_vars - selected_vars)

    f1_score = (2 * true_positive) / (2 * true_positive + false_positive + false_negative) if (2 * true_positive + false_positive + false_negative) > 0 else 0

    perfect_selection = 1.0 if selected_vars == true_vars else 0.0
    
    return f1_score, perfect_selection, selected_vars

def select_best_hyperparameters(X_train, y_train, true_vars, p, domain_range, lam1_values, lam2_values, model_params, epsilon=0.01):
    mse_results = np.zeros((len(lam1_values), len(lam2_values)))
    f1_results = np.zeros((len(lam1_values), len(lam2_values)))
    selection_info = {}

    y_train_mean = np.mean(y_train)
    y_train_std = np.std(y_train)
    
    for i, lam1 in enumerate(lam1_values):
        for j, lam2 in enumerate(lam2_values):
            try:
                train_losses, val_losses, model, l21 = MFDNN(
                    p=p, resp=y_train, func_cov=X_train,
                    num_basis=model_params['num_basis'],
                    layer_sizes=model_params['layer_sizes'],
                    domain_range=domain_range,
                    epochs=model_params['epochs'],
                    val_ratio=model_params['val_ratio'],
                    patience=model_params['patience'],
                    lam1=lam1, lam2=lam2, std_resp=True
                )

                mse_results[i, j] = min(val_losses) if len(val_losses) > 0 else np.mean(train_losses[-10:])

                f1_score, perfect_selection, selected_vars = calculate_selection_metrics(l21, true_vars, epsilon, p)
                f1_results[i, j] = f1_score

                selection_info[f"{i}_{j}"] = {
                    'model': model, 'lam1': lam1, 'lam2': lam2,
                    'f1_score': f1_score, 'mse': mse_results[i, j],
                    'selected_vars': list(selected_vars),
                    'y_mean': y_train_mean, 'y_std': y_train_std,
                    'perfect_selection': perfect_selection
                }
            
                
            except Exception as e:
                mse_results[i, j] = np.inf
                f1_results[i, j] = 0

    best_f1 = np.max(f1_results)
    best_f1_indices = np.where(f1_results == best_f1)
    
    if len(best_f1_indices[0]) > 0:
        
        best_candidates = [selection_info[f"{i}_{j}"] for i, j in zip(best_f1_indices[0], best_f1_indices[1])]
        best_candidate = min(best_candidates, key=lambda x: x['mse'])
    else:
        default_lam1 = 1
        default_lam2 = 0.1
        
        # 创建一个默认的候选者
        best_candidate = {
            'model': None, 'lam1': default_lam1, 'lam2': default_lam2,
            'f1_score': 0, 'mse': np.inf, 'selected_vars': [],
            'y_mean': y_train_mean, 'y_std': y_train_std, 'perfect_selection': 0
        }
    
    return best_candidate['lam1'], best_candidate['lam2'], best_candidate


def evaluate_on_test_set(best_candidate, X_test, y_test, p, domain_range, model_params):
    try:
        y_mean = best_candidate['y_mean']
        y_std = best_candidate['y_std']
        
        test_predictions_normalized = MFDNN_predict(p, best_candidate['model'], X_test, model_params['num_basis'], domain_range)
        test_predictions_original = test_predictions_normalized.detach().numpy() * y_std + y_mean
        test_mse = np.mean((test_predictions_original.flatten() - y_test) ** 2)
        test_rmse = np.sqrt(test_mse)
            
        return test_rmse, best_candidate['f1_score']
    except Exception as e:
        return np.inf, best_candidate['f1_score']


# 运行时间

In [ ]:
results_summary = {}

for T in Ts:
    for p in Ps:
        start_time = time.time()
        print(f"\n{'='*50}")
        print(f"Processing T={T}, p={p}")
        print(f"{'='*50}")

        X_array = np.load(os.path.join(data_path, f"Xlist_T{T}_p{100}.npy"), allow_pickle=True)[0]
        y_array = np.load(os.path.join(data_path, f"ylist_T{T}_p{100}.npy"), allow_pickle=True)[0]

        X_array = X_array[:p, :, :, :]
        p_actual, N, T1, T2 = X_array.shape

        split_idx = N // 2
        X_train = X_array[:, :split_idx, :, :]
        X_test = X_array[:, split_idx:, :, :]

        y_full = y_array  
        y_train = y_full[:split_idx]
        y_test = y_full[split_idx:]

        lam1, lam2, best_candidate = select_best_hyperparameters(
            X_train, y_train, true_vars, p_actual,
            domain_range=[[[0, 0], [1, 1]]] * p_actual,
            lam1_values=lam1_values,
            lam2_values=lam2_values,
            model_params=model_params,
            epsilon=epsilon
        )


        test_rmse, test_f1 = evaluate_on_test_set(
            best_candidate, X_test, y_test, p_actual,
            domain_range=[[[0, 0], [1, 1]]] * p_actual,
            model_params=model_params
        )


        selected_vars = set(best_candidate['selected_vars'])
        elapsed_time = time.time() - start_time

        results_summary[f"T{T}_p{p}"] = {
            'test_rmse': test_rmse,   
            'test_f1': test_f1,
            'selected_vars': selected_vars,
            'run_time_sec': elapsed_time
        }


Processing T=32, p=10
[[0.10559405 0.10300232 0.10725059 0.10681728 0.10757282 0.10905747]
 [0.10987423 0.10851319 0.10378194 0.10753534 0.10932393 0.11161537]
 [0.113538   0.11412442 0.11349944 0.11376411 0.11643638 0.11665644]
 [0.13069622 0.11934952 0.12875655 0.1254214  0.1242499  0.12438659]
 [0.13981645 0.1336454  0.13328725 0.13225207 0.13054368 0.1354917 ]
 [0.14617997 0.14320086 0.14563707 0.13933077 0.14578369 0.147387  ]]
Done T=32, p=10, time=3.77s
  RMSE=0.0190, F1=1.0000, selected_vars={0, 1}

Processing T=32, p=50
[[0.11825228 0.12098156 0.12310185 0.12434211 0.10404489 0.11657661]
 [0.12297417 0.11968825 0.12495811 0.11848837 0.10781882 0.12813686]
 [0.14257886 0.12937066 0.13881984 0.13281831 0.13908277 0.14510238]
 [0.1513679  0.15155737 0.12531415 0.14233902 0.14729644 0.91911381]
 [0.15308598 0.14222987 0.9334985  0.93448335 0.91913605 0.95656079]
 [0.94981635 0.16891864 0.93792498 0.15511987 0.17607351 0.933119  ]]
Done T=32, p=50, time=12.82s
  RMSE=0.0201, F1=1.

# 保存结果

In [ ]:
import pandas as pd
import os

save_path = os.path.join(data_path, "RunTime_mfdnn.csv")


df_results = pd.DataFrame([
    {
        "T": int(key.split("_")[0].replace("T", "")),
        "p": int(key.split("_")[1].replace("p", "")),
        "test_rmse": value["test_rmse"],               
        "test_f1": value["test_f1"],
        "selected_vars": ",".join(map(str, sorted(value["selected_vars"]))),
        "run_time_sec": value["run_time_sec"]
    }
    for key, value in results_summary.items()
])


df_results = df_results.sort_values(by=["T", "p"], ascending=[True, True]).reset_index(drop=True)
df_results.to_csv(save_path, index=False, encoding="utf-8-sig")




✅ Results saved to: /Users/wangdongxue/Documents/MFDNN/MFDNN/RunTimeAnalysis/RunTime_mfdnn.csv
     T    p  test_rmse  test_f1 selected_vars  run_time_sec
0   32   10   0.018961      1.0           0,1      3.770136
1   32   50   0.020120      1.0           0,1     12.821460
2   32  100   0.020719      1.0           0,1     24.314047
3   64   10   0.011068      1.0           0,1     12.713834
4   64   50   0.011786      1.0           0,1     58.636939
5   64  100   0.012707      1.0           0,1    110.480604
6  128   10   0.005704      1.0           0,1     40.897044
7  128   50   0.006030      1.0           0,1    192.179579
8  128  100   0.005955      1.0           0,1    382.274898
